In [6]:
import matplotlib.pyplot as plt
import numpy as np
from simsopt._core import load
from simsopt.geo import plot, MeanSquaredCurvature, CurveSurfaceDistance, CurveCurveDistance, LpCurveCurvature
from simsopt.geo import SurfaceRZFourier, create_equally_spaced_curves, \
    CurveLength, curves_to_vtk, CurveCWSFourier, ArclengthVariation
from simsopt.field import Current, coils_via_symmetries, BiotSavart
from simsopt.objectives import SquaredFlux, QuadraticPenalty
from scipy.optimize import minimize
import os
import sys
import time

from simsopt.configs import get_ncsx_data
from simsopt.field import (BiotSavart, InterpolatedField, coils_via_symmetries, SurfaceClassifier,
                           particles_to_vtk, compute_fieldlines, LevelsetStoppingCriterion, plot_poincare_data)
from simsopt.geo import SurfaceRZFourier, curves_to_vtk
from simsopt.util import in_github_actions, proc0_print, comm_world


[surfaces, coils] = load(f'serial0021326.json')

nphi = 48
ntheta = 48


#only the outer surface
s = SurfaceRZFourier.from_vmec_input('input', range="full torus", nphi=nphi, ntheta=ntheta)


# Plot of plasma over half period with every surface
fig = plot(coils + surfaces , engine="plotly", close=True, show=False)
fig.update_layout(
    autosize=False,
    width=1024,
    height=1024,)
fig.show()



In [4]:
from simsopt.field.magneticfieldclasses import InterpolatedField
import os

def make_Bnormal_plots(bs, s_plot, out_dir='', bs_filename="Bnormal"):
    """
    Plot Bnormal on plasma surface from a MagneticField object.
    Do this quite a bit in the permanent magnet optimization
    and initialization so this is a wrapper function to reduce
    the amount of code.

    Args:
        bs: MagneticField class object.
        s_plot: plasma boundary surface with range = 'full torus'.
        out_dir: Path or string for the output directory for saved files.
        bs_filename: String denoting the name of the output file. 
    """
    nphi = len(s_plot.quadpoints_phi)
    ntheta = len(s_plot.quadpoints_theta)
    bs.set_points(s_plot.gamma().reshape((-1, 3)))
    pointData = {"B_N": np.sum(bs.B().reshape((nphi, ntheta, 3)) * s_plot.unitnormal(), axis=2)[:, :, None]}
    s_plot.to_vtk(out_dir + bs_filename, extra_data=pointData)


def trace_fieldlines(bfield, label, s, out_dir=''):
    """
    Make Poincare plots on a surface as in the trace_fieldlines
    example in the examples/1_Simple/ directory.

    Args:
        bfield: MagneticField or InterpolatedField class object.
        label: Name of the file to write to.
        s: plasma boundary surface.
        out_dir: Path or string for the output directory for saved files.
    """
    from simsopt.field.tracing import compute_fieldlines, \
        plot_poincare_data, \
        IterationStoppingCriterion,LevelsetStoppingCriterion, SurfaceClassifier

    # set fieldline tracer parameters
    nfieldlines = 32
    tmax_fl = 2500
    nplanes = 4

    R0 = np.linspace(s.get_rc(0, 0), s.get_rc(0, 0) + s.get_rc(1, 0) / 4.0, nfieldlines)
    Z0 = np.zeros(nfieldlines)
    #R0 = np.linspace(0.5 * 0.33, 1.4 * 0.33, nfieldlines)
    #Z0 = np.linspace(-0.42 * 0.2, 0.42 * 0.2, nfieldlines)
    phis = [(i / 4) * (2 * np.pi / s.nfp) for i in range(nplanes)]

    print("Compute the fieldlines from the initial locations")
    sc_fieldline = SurfaceClassifier(s, h=0.03, p=2)
    #sc_fieldline.to_vtk(str(out_dir) + 'levelset', h=0.02)

    stopping_crit = [LevelsetStoppingCriterion(sc_fieldline.dist)]

    fieldlines_tys, fieldlines_phi_hits = compute_fieldlines(
        bfield, R0, Z0, tmax=tmax_fl, tol=1e-8,
        phis=phis,
        stopping_criteria=stopping_crit)

    print("Plotting")
    plot_poincare_data(fieldlines_phi_hits, phis, out_dir + label + '.png', dpi=100, surf=s)


# Directory for output
OUT_DIR = "./poincare/"
os.makedirs(OUT_DIR, exist_ok=True)

# calculating magnetic field from coils:
bs = BiotSavart(coils)


n = 64
rs = np.linalg.norm(s.gamma()[:, :, 0:2], axis=2)
zs = s.gamma()[:, :, 2]
r_margin = 0.05
rrange = (np.min(rs) - r_margin, np.max(rs) + r_margin, n)
phirange = (0, 2 * np.pi / s.nfp, n * 2)
zrange = (0, np.max(zs), n // 2)
degree = 4  # 2 is sufficient sometimes
nphi = len(s.quadpoints_phi)
ntheta = len(s.quadpoints_theta)
bs.set_points(s.gamma().reshape((-1, 3)))
Bnormal = np.sum(bs.B().reshape((nphi, ntheta, 3)) * s.unitnormal(), axis=2)
make_Bnormal_plots(bs, s, OUT_DIR, "0_default_biot_savart_pre_poincare_check")

print("Interpolating Field")
bsh = InterpolatedField(
    bs , degree, rrange, phirange, zrange, True, nfp=s.nfp, stellsym=s.stellsym
)
bsh.set_points(s.gamma().reshape((-1, 3)))
trace_fieldlines(bsh, 'poincare_0_default', s, OUT_DIR)

Interpolating Field
Compute the fieldlines from the initial locations
Plotting


<Figure size 640x480 with 0 Axes>